In [56]:
import pandas as pd
import requests
import json
import math
import datetime as dt
import os
from dotenv import load_dotenv

pd.set_option('display.max_rows', 75)


In [57]:
#Client information
load_dotenv()
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [58]:
curDate = dt.datetime.today()
dateString = curDate.strftime('%Y-%m-%d')

In [59]:
#Get authorization token
#Data payload for token request
token_data = {'client_id':client_id,'client_secret':client_secret,'grant_type':'password', 'username':username, 'password':password, 'scope':'api'}

p1 = requests.post('https://scooterscoffee.docebosaas.com/oauth2/token', data=token_data)

p1_result = json.loads(p1.text)

print(p1_result)

{'access_token': 'd8441d3ca713da0ca40c6ba698cc3ed026f643a8', 'expires_in': 3600, 'token_type': 'Bearer', 'scope': 'api', 'refresh_token': '87050f35d76425038866412e4f7b06c4ec58fa6f'}


In [60]:
#Access token information
access_token = p1_result['access_token']

In [61]:
#Headers 
headers = {'Accept':'application/json','Client-ID':client_id, 'Authorization':'Bearer ' + access_token}

In [62]:
#Import API_User_Export file for quicker user lookups
docebo_users_df = pd.read_csv("C:\\Users\\michael.deal\\Downloads\\API_User_Export.csv")

#Set all usernames to lowercase
docebo_users_df['Username'] = docebo_users_df.apply(lambda x: x['Username'].lower(), axis=1)

#Set index to email address
docebo_users_df = docebo_users_df.set_index('Username')

In [63]:
#Functions
LP = ""
dumpText = ""


def batchUpdate(userList: list, learningPlan):
    '''Marks all of the listed users' courses "Completed" in Docebo.
    userList: A list of users to modify
    learningPlan: The name of the learning plan list specified in the script'''
    batch_enroll_url = 'https://scooterscoffee.docebosaas.com/learn/v1/enrollments'

    #Build API loop calls
    for i in userList:
        #Stringify the learning plan
        LP = str(learningPlan)

        #Assemble a payload for each user
        #Create empty dictionary
        dumpText = {}
        dumpText["user_ids"] = [i]
        dumpText["course_ids"] = learningPlan
        dumpText["date_complete"] = dateString
        dumpText["status"] = 2

        #Encode JSON
        payload = json.dumps(dumpText)

        #Put request
        requestPayloadF1 = requests.put(batch_enroll_url, data=payload, headers=headers)

        #Return information
        requestJSONF1 = requestPayloadF1.json()
        print(requestPayloadF1)
        print(dumpText)

    print("Update completed.")


def batchUnenrollByCourse(userList: list, learningPlan):
    '''Unenroll users from specified courses.
    The users identified must be in a list.'''
    batch_enroll_url = 'https://scooterscoffee.docebosaas.com/learn/v1/enrollments'

    #Build API loop calls
    dumpText = {}
    dumpText["user_ids"] = userList
    dumpText["course_ids"] = learningPlan

    #Encode JSON
    payload = json.dumps(dumpText)

    #Put request
    requestPayloadF1 = requests.delete(batch_enroll_url, data=payload, headers=headers)

    #Return information
    requestJSONF1 = requestPayloadF1.json()
    print(requestPayloadF1)
    print(dumpText)

    print("Update completed.")

# Formatting
# {"user_ids":[13085,13088],"course_ids":[197,243],"date_complete":"2023-12-06","status":2}
    

def lookupUserID(userEmail):
    '''Looks up a Docebo user ID based on their email address.'''
    userEmail = userEmail.lower()
    userID = docebo_users_df.loc[userEmail,'User unique ID']
    display(userID)


def lookupStoreUsers(branch, status: str, sortType: str):
    '''Displays the users from a specific store along with their statuses.
    - Branch: Must use four digits (include leading zeroes if necessary)
    - Status options: Active, Inactive, All
    - Sort Type options: Name, Date'''
    store_df = docebo_users_df[docebo_users_df['Branches Codes'] == branch]
    store_df = store_df.reset_index()
    store_df["Full Name"] = store_df.apply(lambda x: str(x['First Name']) + ' ' + str(x['Last Name']), axis=1)
    store_df['Full Name'] = store_df.apply(lambda x: x['Full Name'].title(), axis=1)
    store_df = store_df[['Branches Codes','Full Name','User unique ID','First Name','Last Name','Username','Position','User Creation Date','Deactivated']]
    store_df = store_df.sort_values(by=['Full Name'])
    display_df = pd.DataFrame()
    
    status = status.lower()
    sortType = sortType.lower()

    if status == 'active':
        display_df = store_df[store_df['Deactivated'] == 'No']
        if sortType == 'name':
            display_df = display_df.sort_values(['Full Name'])
        elif sortType == 'date':
            display_df = display_df.sort_values(['User Creation Date','Full Name'])
        else:
            print("Please select a valid option for the sortType.")
        display(display_df)
    elif status == 'inactive':
        display_df = store_df[store_df['Deactivated'] == 'Yes']
        if sortType == 'name':
            display_df = display_df.sort_values(['Full Name'])
        elif sortType == 'date':
            display_df = display_df.sort_values(['User Creation Date','Full Name'])
        else:
            print("Please select a valid option for the sortType.")
        display(display_df)
    elif status == 'all':
        display_df = store_df
        if sortType == 'name':
            display_df = display_df.sort_values(['Full Name'])
        elif sortType == 'date':
            display_df = display_df.sort_values(['User Creation Date','Full Name'])
        else:
            print("Please select a valid option for the sortType.")
        display(display_df)
    else:
        print("Not a valid status code in function.")
    print(list(display_df['User unique ID']))
    print(len(list(display_df['User unique ID'])))

In [64]:
#Get list of All Courses

#API endpoint for getting Courses
courses_URL = 'https://training.scooterscoffee.com/learn/v1/courses'
pageSize = 200

requestPayload1 = requests.get(courses_URL + '?page_size=' + str(pageSize), headers=headers)
requestJSON1 = requestPayload1.json()

#Get total number of users
recordCount = int(requestJSON1['data']['total_count'])

print(requestJSON1['data']['total_count'])

#Calculate the number of loops needed to retrieve all records
loopCounter = recordCount / pageSize

#Round up loopCounter
looper = math.ceil(loopCounter)

262


In [65]:
#Query all possible records for All Courses

#Initialize dataframe
courses_df = pd.DataFrame()

#Initialize page number
pageNum = 1

#Get all pages of data
for i in range(looper):
    requestPayload2 = requests.get(courses_URL + '?page_size=' + str(pageSize) + '&page=' + str(pageNum), headers=headers)
    requestJSON2 = requestPayload2.json()
    print("Page " + str(pageNum) + " has finished its call.")
    pageNum += 1
    itemsDict = requestJSON2['data']['items']
    itemsDict_df = pd.DataFrame.from_dict(itemsDict)
    courses_df = pd.concat([courses_df, itemsDict_df], ignore_index=True)
    
print("All records pulled.")

#Expand out items from "category" field to get appropriate courses
#Category ID
courses_df['category_id'] = courses_df.apply(lambda x: x['category']['id'], axis=1)
#Category Name
courses_df['category_name'] = courses_df.apply(lambda x: x['category']['name'], axis=1)

Page 1 has finished its call.
Page 2 has finished its call.
All records pulled.


In [66]:
#Ad hoc
#Import list of completed checklist users
cl_complete_df = pd.read_csv("C:\\Users\\michael.deal\\OneDrive - Boundless Enterprises, LLC\\Documents\\Docebo\\Reports\\Checklists\\completed_users.csv", dtype={'USER_ID':'str'})

cl_list = list(cl_complete_df['USER_ID'])


In [67]:
#Simplify courses_df to needed data
courses_simple_df = courses_df[['category_name','name','id_course']]

courses_simple_df = courses_simple_df.sort_values(by=['category_name','name','id_course'])

In [68]:
#Lists of courses (by ID number) by learning plan that can be updated from SCTC carryover. To build lists, you need to visually inspect the courses_simplify_df dataframe and pick out courses that were carried over from SCTC.

# Barista courses
b_old = [16, 19, 13, 22, 18, 17, 20, 21, 10, 15, 9, 14]
b_new = [23, 16, 19, 13, 22, 18, 17, 116, 20, 21, 117, 104, 10, 15, 9, 14]

#Junior Manager courses
jm_old = [60, 61, 62, 63, 64, 65]
jm_new = [60, 61, 62, 63, 64, 65]

#Store Manager courses
sm_old = [53, 45, 54, 51, 48, 49, 42, 50, 46, 44, 47, 52, 103, 41]
sm_new = [238, 53, 45, 54, 51, 48, 49, 42, 50, 46, 44, 47, 52, 103, 43, 41]

#Manager (Barista Skills) courses
mbs_old = [74, 70, 77, 66, 71, 73, 69, 67, 78, 75, 79, 76]
mbs_new = [74, 70, 77, 66, 71, 73, 241, 69, 67, 242, 240, 78, 75, 79, 76]

#Franchise Owner courses
fo_old = [56, 32, 35, 33, 40, 39, 36, 34, 38, 37, 28, 31, 29, 30]
fo_new = [56, 32, 35, 33, 40, 39, 36, 34, 38, 37, 28, 31, 29, 30]

#Corp courses
corp_test = [197, 243]

#Store Manager Certification
sm_cert = [284]

#Barista checklist
bar_check = [23]

In [69]:
# #Looks up a user ID from the imported file based on their email.
# lookupUserID('emmar.archoldings@gmail.com')

In [70]:
# #Looks up the users in a store 
# lookupStoreUsers('0070', 'active', 'date')

In [71]:
# #Updates all the courses for the selected users in the specified learning plan.
# batchUpdate([00001], bar_check)

In [72]:
# #Unenrolls the users in the list from the specified learning plan.
# batchUnenrollByCourse([27547], sm_cert)